In [1]:
# Loading and displaying an image
# from PIL import Image
# img = Image.open('/home/dvveera/db/orl/s1/1.pgm')
# print(img.format, img.size, img.mode)
# display(img)

In [2]:
# Load the dataset
import load_dataset
import numpy as np
import math
from PIL import Image

dir_src = '/home/dvveera/dvveera_hdd2tb/orl'
data_label = load_dataset.load_data_label(dir_src)

# Training, validation and test data with labels
# data_label = list(train, valid, test, mean_train)
train_storage, train_labels = zip(*data_label[0])
valid_storage, valid_labels = zip(*data_label[1])
test_storage, test_labels = zip(*data_label[2])
mean_storage = data_label[3]

Assigned Labels to Training, Validation and Test Data
Train data: 20/240
Train data: 40/240
Train data: 60/240
Train data: 80/240
Train data: 100/240
Train data: 120/240
Train data: 140/240
Train data: 160/240
Train data: 180/240
Train data: 200/240
Train data: 220/240
Valid data: 20/80
Valid data: 40/80
Valid data: 60/80
Test data: 20/80
Test data: 40/80
Test data: 60/80


In [3]:
# Sample set and labels
# import numpy as np
# from PIL import Image
# train_storage = []
# train_storage.append(np.array([[196, 35, 234], [232, 59, 244], [243, 57, 226]]))
# train_storage.append(np.array([[188, 15, 236], [244, 44, 228], [251, 48, 230]]))
# train_storage.append(np.array([[246, 48, 222], [225, 40, 226], [208, 35, 234]]))
# train_storage.append(np.array([[208, 16, 235], [255, 44, 229], [236, 34, 247]]))
# train_storage.append(np.array([[245, 21, 213], [254, 55, 252], [215, 51, 249]]))
# train_storage.append(np.array([[248, 22, 225], [252, 30, 240], [242, 27, 244]]))
# train_storage.append(np.array([[255, 223, 224], [255, 0, 255], [249, 255, 235]]))
# train_storage.append(np.array([[234, 255, 205], [251, 0, 251], [238, 253, 240]]))
# train_storage.append(np.array([[232, 255, 231], [247, 38, 246], [190, 236, 250]]))
# train_storage.append(np.array([[255, 241, 208], [255, 28, 255], [194, 234, 188]]))
# train_storage.append(np.array([[237, 243, 237], [237, 19, 251], [227, 225, 237]]))
# train_storage.append(np.array([[224, 251, 215], [245, 31, 222], [233, 255, 254]]))
# train_labels = [2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1]
# for i in train_storage:
#     dis1 = Image.fromarray(np.transpose(np.uint8(i)))
#     dis1 = dis1.resize((100, 100))
#     display(dis1)
# print(train_storage)

In [4]:
%%time
# Linear Discriminant Analysis
from collections import Counter
# Converting images to a one-dimensional vector
train_1d = [t.flatten() for t in train_storage]
# Calculate the mean of images in each class
label_dict = Counter(train_labels)
mean = {}
for idx, t in enumerate(train_1d):
    lab = train_labels[idx]
    if lab not in mean:
        mean[lab] = t / label_dict[lab]
    else:
        mean[lab] += t / label_dict[lab]
# print('Calculated the means of the images in each class')
# print(mean)

CPU times: user 5.82 ms, sys: 192 µs, total: 6.01 ms
Wall time: 5.87 ms


In [5]:
%%time
# Calculate in class scatter matrix
Sw = []
for idx, t in enumerate(train_1d):
    lab = train_labels[idx]
    if idx == 0:
        Sw = np.outer(t - mean[lab], t - mean[lab])
    else:
        Sw += np.outer(t - mean[lab], t - mean[lab])
print('Calculated the in-class scatter matrix')
# print(Sw)

Calculated the in-class scatter matrix
CPU times: user 36.2 s, sys: 40.4 s, total: 1min 16s
Wall time: 1min 16s


In [6]:
%%time
# Calculate between class scatter matrix
# Calculate the mean of all images
mean_all = np.sum(train_1d, axis = 0) / len(train_1d)
# Calculate between class scatter matrix
for idx, m in enumerate(mean):
    if idx == 0:
        Sb = label_dict[m] * np.outer(mean[m] - mean_all, mean[m] - mean_all)
    else:
        Sb += label_dict[m] * np.outer(mean[m] - mean_all, mean[m] - mean_all)
print('Calculated the between-class scatter matrix')
# print(Sb)

Calculated the between-class scatter matrix
CPU times: user 9.26 s, sys: 12.8 s, total: 22.1 s
Wall time: 22.1 s


In [7]:
%%time
# Compute the generalized eigen values and eigen vectors
A = np.dot(np.linalg.inv(Sw), Sb)
eig_values, eig_vectors = np.linalg.eigh(A)
# print(eig_values)
# print(mean_all)
# print(mean)
eig = list(zip(eig_values, np.transpose(eig_vectors)))

CPU times: user 5min 26s, sys: 1.98 s, total: 5min 28s
Wall time: 1min 21s


In [8]:
%%time
# Sort eigenvalues and eigenvectors
eig = sorted(eig, key=lambda x: x[0], reverse = True)
print('Sorted eigen values and eigen vectors from high to low')
# print(eig)
# temp = eig_values
# sorted(temp, reverse = True)
# print(temp)

Sorted eigen values and eigen vectors from high to low
CPU times: user 765 µs, sys: 113 µs, total: 878 µs
Wall time: 877 µs


In [9]:
%%time
# Select the first C - 1 eigen vectors
eig_values, eig_vectors = zip(*eig)
new_eig = []
for idx, e in enumerate(eig):
    if idx < (len(label_dict) - 1):
        new_eig.append(e)
print('Selected the first C - 1 eigenvectors')
# print(new_eig)
eig_values, eig_vectors = zip(*new_eig)

Selected the first C - 1 eigenvectors
CPU times: user 14.2 ms, sys: 4.32 ms, total: 18.5 ms
Wall time: 18.4 ms


In [ ]:
# # Select specified number of eigenvectors
# n_eig = int(np.ceil(0.01 * len(new_eig)))
# temp_values = []
# temp_vectors = []
# for i in range(n_eig):
#     temp_values.append(eig_values[i])
#     temp_vectors.append(eig_vectors[i])
# eig_values, eig_vectors = temp_values, temp_vectors
# print(len(eig_values))

In [ ]:
# # Remove last 40% of eigenvectors
# n_eig = int(np.ceil(0.6 * len(eig_values)))
# temp_values = []
# temp_vectors = []
# for i in range(n_eig):
#     temp_values.append(eig_values[i])
#     temp_vectors.append(eig_vectors[i])
# eig_values, eig_vectors = temp_values, temp_vectors
# print(len(eig_values))

In [ ]:
# # Select eigenvectors based on energy
# threshold = 0.9
# sum_eig = np.sum(eig_values)
# temp_values = []
# temp_vectors = []
# sum_e = 0
# for e in eig_values:
#     if (sum_e / sum_eig) > threshold:
#         break
#     else:
#         temp_values.append(e)
#         idx = eig_values.index(e)
#         temp_vectors.append(eig_vectors[idx])
#     sum_e += e
# eig_values, eig_vectors = temp_values, temp_vectors
# print(len(eig_values))

In [ ]:
# # Select eigenvectors based on stretching
# threshold = 0.01
# largest_eig = eig_values[0]
# temp_values = []
# temp_vectors = []
# for e in eig_values:
#     if (e / largest_eig) < threshold:
#         break
#     else:
#         temp_values.append(e)
#         idx = eig_values.index(e)
#         temp_vectors.append(eig_vectors[idx])
# eig_values, eig_vectors = temp_values, temp_vectors
# print(len(eig_values))

In [ ]:
# # Remove the first eigenvector
# temp_values = []
# temp_vectors = []
# first_eig = eig_values[0]
# for e in eig_values:
#     if e == first_eig:
#         pass
#     else:
#         temp_values.append(e)
#         idx = eig_values.index(e)
#         temp_vectors.append(eig_vectors[idx])
# eig_values, eig_vectors = temp_values, temp_vectors
# print(len(eig_values))

In [10]:
%%time
# Create projection matrix
# eig_values, eig_vectors = zip(*new_eig)
V = np.transpose(eig_vectors)

CPU times: user 4.94 ms, sys: 139 µs, total: 5.08 ms
Wall time: 4.51 ms


In [11]:
%%time
# Project training images into subspace
data_matrix = np.transpose(train_1d)
proj_data_matrix = np.dot(eig_vectors, data_matrix)

CPU times: user 14.3 ms, sys: 9.5 ms, total: 23.8 ms
Wall time: 14 ms


In [12]:
%%time
# Identify test images
valid_1d = [t.flatten() for t in valid_storage]
# Create data matrix
valid_data_matrix = np.transpose(valid_1d)
# Project test images into the same subspace as training images
proj_valid_matrix = np.dot(eig_vectors, valid_data_matrix)

CPU times: user 5.53 ms, sys: 4.23 ms, total: 9.76 ms
Wall time: 7.3 ms


In [13]:
%%time
# Calculate L2 norms
tr1 = np.transpose(proj_valid_matrix)
tr2 = np.transpose(proj_data_matrix)
l2 = []
for a in tr1:
    l1 = [np.linalg.norm(a - b) for b in tr2]
    l2.append(l1)
# Find the index of the training image that matches closely with the test image
idx = [l.index(min(l)) for l in l2]

CPU times: user 123 ms, sys: 17.8 ms, total: 141 ms
Wall time: 95.1 ms


In [ ]:
# # Calculate L1 norms
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [np.sum(abs(a - b)) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(min(l)) for l in l2]

In [ ]:
# # Calculate covariance measure
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [ - np.dot(a / np.linalg.norm(a), b / np.linalg.norm(b)) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(min(l)) for l in l2]

In [ ]:
# Function to calculate mahalanobis distance between two image vectors
def mah(x, y):
    m = 0
    for i in range(len(x)):
        m += x[i] * y[i] * 1 / np.sqrt(eig_values[i])
    return -m

In [ ]:
# # Calculate mahalanobis
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [mah(a, b) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(min(l)) for l in l2]

In [ ]:
# Function to calculate correlation between two image vectors
def corr(x, y):
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    std_x = np.std(x, ddof = 1)
    std_y = np.std(y, ddof = 1)
    m = 0
    for i in range(len(x)):
        m += (x[i] - mean_x) * (y[i] - mean_y) / (std_x * std_y)
    return m

In [ ]:
# # Calculate correlation
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [corr(a, b) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(max(l)) for l in l2]

In [14]:
%%time
# Find the label for the test images
out = []
for i in idx:
    out.append(train_labels[i])
# print(list(zip(out, idx)))
# print(list(zip(valid_labels, idx)))

CPU times: user 11 µs, sys: 1e+03 ns, total: 12 µs
Wall time: 13.8 µs


In [15]:
%%time
# Find the accuracy
correct = 0
for i in range(len(out)):
    if out[i] == valid_labels[i]:
        correct += 1
accuracy = correct * 100 / len(out)
print('Accuracy(%): ' + str(accuracy))

Accuracy(%): 96.25
CPU times: user 325 µs, sys: 48 µs, total: 373 µs
Wall time: 238 µs


In [ ]:
# Identify test images
test_1d = [t.flatten() for t in test_storage]
# Create data matrix
test_data_matrix = np.transpose(test_1d)
# Project test images into the same subspace as training images
proj_test_matrix = np.dot(eig_vectors, test_data_matrix)

In [ ]:
# Calculate L2 norms
tr1 = np.transpose(proj_test_matrix)
tr2 = np.transpose(proj_data_matrix)
l2 = []
for a in tr1:
    l1 = [np.linalg.norm(a - b) for b in tr2]
    l2.append(l1)
# Find the index of the training image that matches closely with the test image
idx = [l.index(min(l)) for l in l2]

In [ ]:
# # Calculate L1 norms
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [np.sum(abs(a - b)) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(min(l)) for l in l2]

In [ ]:
# # Calculate covariance measure
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [ - np.dot(a / np.linalg.norm(a), b / np.linalg.norm(b)) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(min(l)) for l in l2]

In [ ]:
# # Calculate mahalanobis
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [mah(a, b) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(min(l)) for l in l2]

In [ ]:
# # Calculate correlation
# tr1 = np.transpose(proj_valid_matrix)
# tr2 = np.transpose(proj_data_matrix)
# l2 = []
# for a in tr1:
#     l1 = [corr(a, b) for b in tr2]
#     l2.append(l1)
# # Find the index of the training image that matches closely with the test image
# idx = [l.index(max(l)) for l in l2]

In [ ]:
# Find the label for the test images
out = []
for i in idx:
    out.append(train_labels[i])

In [ ]:
# Find the accuracy
correct = 0
for i in range(len(out)):
    if out[i] == test_labels[i]:
        correct += 1
accuracy = correct * 100 / len(out)
print('Accuracy(%): ' + str(accuracy))

In [ ]:
# # Print images (true / predicted)
# j = 0
# for i in range(len(out)):
#     dis1 = Image.fromarray(train_storage[idx[i]])
#     display(dis1)
#     print('Predicted:' + str(out[i]))
#     dis2 = Image.fromarray(test_storage[j])
#     display(dis2)
#     print('True:' + str(test_labels[j]))
#     print('-----------------------------------')
#     j += 1
# # print(list(zip(out, idx)))
# # print(list(zip(test_labels, idx)))